# LangChain과 Gemini를 활용한 데이터 분석 에이전트

이 노트북에서는 LangChain과 Gemini를 활용한 데이터 분석 에이전트의 구현 및 사용 방법을 설명합니다.

In [9]:
%pip install langgraph

Note: you may need to restart the kernel to use updated packages.


In [10]:
%pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langgraph.graph import StateGraph

print('LangGraph successfully installed!')

LangGraph successfully installed!


In [8]:
# 나는 가난뱅이니까 무료 api를 제공하는 gemini를 사용한다. 
%pip install langchain-google-genai python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [12]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

# .env 파일에서 환경 변수 로드
load_dotenv()

# Google Gemini 설정
gemini = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite"
)

# Google Gemini 상
gemini_response = gemini.invoke('박대원이 2년 안에 삼일회계법인에 입사할 수 있을지 예측하는 모델을 제시하시오.')
print('마법의 소라고동께서 가로되', gemini_response)

마법의 소라고동께서 가로되 content='## 박대원의 삼일회계법인 입사 가능성 예측 모델\n\n박대원이 2년 안에 삼일회계법인(이하 삼일)에 입사할 수 있을지 예측하기 위해 다양한 요소를 고려한 모델을 제시합니다. 이 모델은 다음과 같은 단계로 구성됩니다.\n\n**1. 데이터 수집 및 분석:**\n\n*   **필요 데이터:**\n    *   **박대원 관련 데이터:**\n        *   학력: 대학/전공 (경영, 회계, 경제 등 관련 전공 우대), 학점, 졸업 예정일\n        *   자격증: CPA, AICPA, 기타 관련 자격증 (IFRS 전문가 등) 취득 여부 및 취득 시점\n        *   경험: 인턴십 (삼일, 기타 회계법인, 관련 기업), 아르바이트, 동아리 활동, 리더십 경험, 봉사활동 등\n        *   어학 능력: 토익, 토플, 오픽, 토익스피킹 점수 (영어 능력 중요)\n        *   면접 준비: 스터디 참여 여부, 모의 면접 경험, 면접 예상 질문 대비 수준\n        *   자기소개서: 작성 경험, 피드백 반영 여부, 완성도\n        *   기타: 개인적인 강점, 특기, 삼일 입사에 대한 열정 등\n    *   **삼일 관련 데이터:**\n        *   채용 공고 분석: 채용 규모, 채용 시기, 채용 기준 변화 추이\n        *   합격자 데이터 분석: 합격자들의 학력, 자격증, 경험, 어학 능력 등 (익명화된 데이터 활용)\n        *   업계 동향: 회계 감사 시장의 변화, 삼일의 경쟁 환경 (딜로이트, EY, KPMG, PwC)\n        *   경제 상황: 국내외 경제 상황 (취업 시장에 미치는 영향)\n\n*   **데이터 분석 방법:**\n    *   **기술 통계 분석:** 각 변수별 평균, 표준편차, 빈도 분석 등을 통해 기본적인 특징 파악\n    *   **상관관계 분석:** 각 변수 간의 상관관계 분석 (예: 학점과 CPA 합격 간의 상관

## 1. 기본 구성 및 환경 설정

필요한 라이브러리를 임포트하고 기본 설정을 구성합니다.

In [ ]:
# 필요한 라이브러리 임포트
import os
from typing import List, Dict, Any

import streamlit as st
import google.generativeai as genai
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import AIMessage, HumanMessage
from langchain_mcp_adapters import JupyterMCPServer, MetadataMCPServer

## 2. API 키 및 의존성 설정

Gemini API 키를 설정하고 필요한 환경 변수를 구성합니다.

In [ ]:
# Gemini API 키 설정
os.environ["GOOGLE_API_KEY"] = "your-api-key"  # 실제 API 키로 교체하세요
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

# MCP 서버 초기화
jupyter_server = JupyterMCPServer()
metadata_server = MetadataMCPServer()

## 3. 데이터 분석 에이전트 구현

DataAnalysisAgent 클래스를 구현하여 LLM, 도구, 메모리를 통합합니다.

In [ ]:
class DataAnalysisAgent:
    def __init__(self):
        # 도구 설정
        self.tools = [
            Tool(
                name="jupyter_execute",
                func=jupyter_server.execute,
                description="주피터 노트북에서 코드를 실행합니다."
            ),
            Tool(
                name="get_metadata",
                func=metadata_server.get_metadata,
                description="팀 메타데이터를 조회합니다."
            )
        ]
        
        # 메모리 초기화
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )
        
        # Gemini 모델 초기화
        self.model = genai.GenerativeModel('gemini-2.0-flash-lite')
        
        # Agent 설정
        self.agent = AgentExecutor(
            llm=self.model,
            tools=self.tools,
            memory=self.memory,
            verbose=True
        )

    def analyze(self, query: str) -> str:
        try:
            result = self.agent.run(query)
            return result
        except Exception as e:
            return f"분석 중 오류가 발생했습니다: {str(e)}"

## 4. 도구 및 메모리 사용 예제

에이전트를 생성하고 간단한 분석 작업을 수행해봅니다.

In [ ]:
# 에이전트 생성
agent = DataAnalysisAgent()

# 샘플 데이터 분석 요청
query = "sales_data.csv 파일의 처음 5행을 보여주고 기본 통계 정보를 분석해주세요."
result = agent.analyze(query)
print(result)

## 5. 유틸리티 함수 구현

에러 처리와 결과 포맷팅을 위한 유틸리티 함수를 구현합니다.

In [ ]:
def format_analysis_result(result: str) -> str:
    """분석 결과를 포맷팅하는 유틸리티 함수"""
    try:
        # 결과 포맷팅 로직 구현
        formatted_result = f"분석 결과:\n{'-' * 50}\n{result}\n{'-' * 50}"
        return formatted_result
    except Exception as e:
        return f"결과 포맷팅 중 오류 발생: {str(e)}"

def log_error(error: Exception, query: str) -> None:
    """에러 로깅 유틸리티 함수"""
    print(f"에러 발생 - 쿼리: {query}")
    print(f"에러 메시지: {str(error)}")
    # 추가적인 로깅 로직 구현

## 6. 테스트 및 결과 확인

구현된 에이전트의 동작을 테스트하고 결과를 확인합니다.

In [ ]:
# 테스트 쿼리 실행
test_queries = [
    "매출 데이터의 월별 트렌드를 분석해주세요.",
    "고객 세그먼트별 구매 패턴을 시각화해주세요.",
    "제품 카테고리별 판매량 TOP 5를 보여주세요."
]

for query in test_queries:
    try:
        result = agent.analyze(query)
        formatted_result = format_analysis_result(result)
        print(formatted_result)
    except Exception as e:
        log_error(e, query)